In [1]:
# PREPROCESSING NOTEBOOK
import torch
import clip
from PIL import Image
from pathlib import Path

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50", device=device)
image_dir = Path('./images')
extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.webp'}

# 1. Collect Paths
image_paths = sorted([
    p for p in image_dir.glob('**/*') 
    if p.is_file() and p.suffix.lower() in extensions
])

# 2. Process Images in Batches (to avoid memory crashes)
print(f"Processing {len(image_paths)} images...")
all_features = []

with torch.no_grad():
    for path in image_paths:
        img = preprocess(Image.open(path).convert("RGB")).unsqueeze(0).to(device)
        feat = model.encode_image(img)
        feat /= feat.norm(dim=-1, keepdim=True) # Normalize
        all_features.append(feat)

# 3. Combine and Save
image_features = torch.cat(all_features)
save_data = {
    'features': image_features.cpu(), # Save as CPU tensor for compatibility
    'paths': [str(p) for p in image_paths]
}

torch.save(save_data, 'image_embeddings.pt')
print("Successfully saved embeddings to image_embeddings.pt")

Processing 1972 images...
Successfully saved embeddings to image_embeddings.pt
